In [ ]:
print("Installing required libraries: earthengine-api, pandas, scikit-learn, rasterio, folium...")
!pip install earthengine-api pandas scikit-learn rasterio folium --quiet
print("Installation complete.")


Installing required libraries: earthengine-api, pandas, scikit-learn, rasterio, folium...
Installation complete.


In [ ]:
import ee
import time
import os
import pandas as pd
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling
import folium
import folium.raster_layers
from folium.plugins import MarkerCluster
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import json
from google.colab import files
from google.colab import drive

print("Libraries imported successfully.")


Libraries imported successfully.


In [ ]:
try:
    # Pass the project ID directly to ee.Initialize()
    ee.Initialize(project='geohack-475905')
    print("GEE authenticated successfully.")
except Exception as e:
    print(f"GEE authentication failed: {e}")
    print("Please try authenticating manually:")
    ee.Authenticate()
    ee.Initialize(project='geohack-475905')
    print("GEE authenticated successfully after manual step.")

# 2. Mount Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")



GEE authenticated successfully.
Mounted at /content/drive
Google Drive mounted successfully.


In [ ]:
print("Starting Part 1: GEE Data Export...")

# --- Configuration ---
# All files will be saved in this folder in your Google Drive
DRIVE_FOLDER = 'wildfire_hackathon_pro'
CSV_FILENAME = 'training_data_india'
TIF_FILENAME = 'prediction_features_india'

# Full paths for later use
DRIVE_PATH = f'/content/drive/My Drive/{DRIVE_FOLDER}'
CSV_PATH = f'{DRIVE_PATH}/{CSV_FILENAME}.csv'
TIF_PATH = f'{DRIVE_PATH}/{TIF_FILENAME}.tif'

# Create the folder if it doesn't exist
if not os.path.exists(DRIVE_PATH):
    os.makedirs(DRIVE_PATH)

# --- 1. Define Datasets & Area of Interest (Python Syntax) ---
aoi = ee.Geometry.Rectangle([
  68.11, 6.75, 97.41, 35.67
])
startDate = '2020-01-01'
endDate = '2020-12-31'

# a) Burned Area
modisBurn = ee.ImageCollection('MODIS/006/MCD64A1').filterDate(startDate, endDate).select('BurnDate')
burnedImage = modisBurn.max().gt(0).unmask(0).rename('burned')

# b) Topography
elevation = ee.Image('WWF/HydroSHEDS/03CONDEM').select('b1').rename('elevation')
terrain = ee.Algorithms.Terrain(elevation)
slope = terrain.select('slope').rename('slope')
aspect = terrain.select('aspect').rename('aspect')

# c) Vegetation (Original Fuel Type)
landcover = ee.ImageCollection('MODIS/006/MCD12Q1').filterDate(startDate, endDate).select('LC_Type1').first().rename('fuel_type')

# d) Climate
modisTemp = ee.ImageCollection('MODIS/006/MOD11A1').filterDate(startDate, endDate).select('LST_Day_1km')
meanTemp = modisTemp.mean().multiply(0.02).subtract(273.15).rename('mean_temp')
modisSolar = ee.ImageCollection("MODIS/006/MOD09GA").filterDate(startDate, endDate).select('sur_refl_b01')
solarRad = modisSolar.mean().rename('solar_rad')
precip = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD").filterDate(startDate, endDate).select('precipitation')
totalPrecip = precip.sum().rename('total_precip')

# e) Canopy Height (Proxy from Landcover)
modisLC = ee.ImageCollection('MODIS/006/MCD12Q1').filterDate(startDate, endDate).select('LC_Type1').first()
# Remap land cover classes. Forest types (1-5) become 1, others become 0.
canopyHeight = modisLC.remap(
  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], # From classes
  [1, 1, 1, 1, 1, 0, 0, 0, 0, 0,  0,  0,  0,  0,  0,  0,  0]  # To classes
).rename('canopy_height')

# --- 2. Assemble Feature Stack ---
projection = elevation.projection().atScale(1000)
featureStack = ee.Image.cat([
  slope,
  aspect,
  landcover,  # This is the original fuel_type
  meanTemp,
  totalPrecip,
  solarRad,
  canopyHeight # This is the new binary canopy band
]).rename([
  'slope',
  'aspect',
  'fuel_type',
  'mean_temp',
  'total_precip',
  'solar_rad',
  'canopy_height'
]).reproject(projection).clip(aoi).float() # Cast to float

finalImage = featureStack.addBands(burnedImage).float()

# --- 3. Define Export Tasks ---
print("Defining GEE export tasks...")

# Task 1: Export Training Data (CSV)
trainingData = finalImage.sample(
  region=aoi,
  scale=1000,
  numPixels=5000,
  geometries=True
)
task_csv = ee.batch.Export.table.toDrive(
  collection=trainingData,
  description='export_training_data_india_csv',
  folder=DRIVE_FOLDER,
  fileNamePrefix=CSV_FILENAME,
  fileFormat='CSV'
)

# Task 2: Export Prediction Features (GeoTIFF)
task_tif = ee.batch.Export.image.toDrive(
  image=featureStack,
  description='export_prediction_features_india_tif',
  folder=DRIVE_FOLDER,
  fileNamePrefix=TIF_FILENAME,
  scale=1000,
  region=aoi,
  maxPixels=1e10
)

# --- 4. Start Exports ---
task_csv.start()
task_tif.start()

print(f"GEE export tasks started. Files will be saved to '{DRIVE_PATH}'")
print("This will take several minutes. Please wait for Cell 5 to complete.")

# %%
# --- Cell 5: Wait for GEE Tasks to Complete ---

print("Waiting for GEE tasks to complete. This can take 5-15 minutes...")
print("You can check task status at: https://code.earthengine.google.com/tasks")

while task_csv.active() or task_tif.active():
    # Print status every 30 seconds
    csv_status = task_csv.status()['state']
    tif_status = task_tif.status()['state']
    print(f"Status: CSV Export ({csv_status}), TIF Export ({tif_status})")

    if csv_status == 'FAILED' or tif_status == 'FAILED':
        print("Error: A GEE task failed. Check the GEE Tasks tab for details.")
        task_csv.cancel()
        task_tif.cancel()
        break

    time.sleep(30)

if task_csv.status()['state'] == 'COMPLETED' and task_tif.status()['state'] == 'COMPLETED':
    print("SUCCESS: GEE export tasks completed!")
    print(f"Files saved to: {DRIVE_PATH}")
else:
    print("Error: Tasks did not complete successfully.")



Starting Part 1: GEE Data Export...
Defining GEE export tasks...
GEE export tasks started. Files will be saved to '/content/drive/My Drive/wildfire_hackathon_pro'
This will take several minutes. Please wait for Cell 5 to complete.
Waiting for GEE tasks to complete. This can take 5-15 minutes...
You can check task status at: https://code.earthengine.google.com/tasks
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (READY), TIF Export (READY)
Status: CSV Export (COMPLETED), TIF Export (RUNNING)
Status: CSV Export (COMPLETED), TIF Export (RUNNING)
Status: CSV Export (COMPLETED), TIF Export (RUNNING)
Status: CSV Export (COMPLETED), TIF Export (RUNNING)
Status: CSV Export (COMPLETED), TIF Export (RUNNING)
Status: CSV Export (COMPLETED), TIF Ex

In [ ]:
print("Starting Part 2: ML Model Training...")
# Load data directly from the Google Drive path
print(f"Loading training data from: {CSV_PATH}")

if not os.path.exists(CSV_PATH):
    print(f"Error: {CSV_PATH} not found. Did Cell 5 complete successfully?")
else:
    df = pd.read_csv(CSV_PATH)
    df = df.fillna(0) # Simple fillna

    FEATURES = [
        'slope', 'aspect', 'fuel_type', 'mean_temp',
        'total_precip', 'solar_rad', 'canopy_height'
    ]
    TARGET = 'burned'

    missing_cols = [col for col in FEATURES + [TARGET] if col not in df.columns]
    if missing_cols:
        print(f"Error: Missing columns in training_data.csv: {missing_cols}")
    else:
        X = df[FEATURES]
        y = df[TARGET]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        print("Training RandomForest model...")
        model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        model.fit(X_train, y_train)

        print("Evaluating model...")
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model Accuracy: {accuracy * 100:.2f}%")
        print(classification_report(y_test, y_pred))


Starting Part 2: ML Model Training...
Loading training data from: /content/drive/My Drive/wildfire_hackathon_pro/training_data_india.csv
Training RandomForest model...
Evaluating model...
Model Accuracy: 99.14%
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       919
         1.0       0.00      0.00      0.00         8

    accuracy                           0.99       927
   macro avg       0.50      0.50      0.50       927
weighted avg       0.98      0.99      0.99       927



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("Generating risk map...")
# Make these global
risk_map = None
map_bounds = None

if not os.path.exists(TIF_PATH):
    print(f"Error: {TIF_PATH} not found. Did Cell 5 complete successfully?")
elif 'model' not in locals():
    print("Error: Model was not trained. Skipping prediction.")
else:
    print(f"Loading prediction features from: {TIF_PATH}")
    with rasterio.open(TIF_PATH) as src:
        img = src.read()
        profile = src.profile
        img = np.nan_to_num(img, nan=0)

        n_bands, height, width = img.shape
        img_reshaped = img.reshape(n_bands, -1)
        img_transposed = img_reshaped.T

        print("Predicting risk for all pixels...")
        risk_probabilities = model.predict_proba(img_transposed)[:, 1]
        risk_map = risk_probabilities.reshape(height, width)

        # Get bounds for Folium
        map_bounds = [[src.bounds.bottom, src.bounds.left], [src.bounds.top, src.bounds.right]]
        print(f"Map bounds calculated: {map_bounds}")
        print("Risk map generated!")


Generating risk map...
Loading prediction features from: /content/drive/My Drive/wildfire_hackathon_pro/prediction_features_india.tif
Predicting risk for all pixels...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Map bounds calculated: [[6.747869957394766, 68.1073348516741], [36.57193739016288, 97.41037941965288]]
Risk map generated!


In [ ]:
print("Creating the master Folium map...")

# --- A. Define Helper Functions ---
def add_live_fires(m):
    print("Adding live fires layer...")
    try:
        fires_url = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_Global_24h.csv'
        df_fires = pd.read_csv(fires_url)
        if df_fires.empty:
            print("No live fires detected globally.")
            return

        fire_cluster = MarkerCluster(name="Live Fires (Global, 24h)")
        for idx, row in df_fires.iterrows():
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                tooltip=f"Fire (Confidence: {row['confidence']}%)",
                icon=folium.Icon(color='red', icon='fire', prefix='fa')
            ).add_to(fire_cluster)
        fire_cluster.add_to(m)
        print(f"Added {len(df_fires)} live fire points globally.")
    except Exception as e:
        print(f"Warning: Could not load live fire data. {e}")

def add_pois(m):
    print("Adding POIs layer...")
    poi_data = {
      "type": "FeatureCollection",
      "features": [
        {"type": "Feature", "geometry": {"type": "Point", "coordinates": [-119.7871, 36.7378]}, "properties": {"name": "Hospital: Community Regional (USA)", "type": "Hospital"}},
        {"type": "Feature", "geometry": {"type": "Point", "coordinates": [77.2167, 28.6139]}, "properties": {"name": "Shelter: India Gate (IND)", "type": "Shelter"}},
        {"type": "Feature", "geometry": {"type": "Point", "coordinates": [78.0422, 27.1751]}, "properties": {"name": "Evacuation Point: Taj Mahal (IND)", "type": "School"}}
      ]
    }
    def get_poi_icon(feature_type):
        if feature_type == 'Hospital': return folium.Icon(color='blue', icon='hospital', prefix='fa')
        elif feature_type == 'Shelter': return folium.Icon(color='green', icon='home', prefix='fa')
        else: return folium.Icon(color='purple', icon='school', prefix='fa')

    poi_feature_group = folium.FeatureGroup(name="Points of Interest")
    for feature in poi_data['features']:
        coords = feature['geometry']['coordinates']
        props = feature['properties']
        folium.Marker(
            location=[coords[1], coords[0]], # GeoJSON is (lon, lat)
            tooltip=f"{props['type']}: {props['name']}",
            icon=get_poi_icon(props['type'])
        ).add_to(poi_feature_group)
    poi_feature_group.add_to(m)

# --- B. Create the Base Map ---
m = folium.Map(location=[20, 0], zoom_start=2, tiles="CartoDB positron", attribution_control=False)

# --- C. Add the ML Risk Layer (for India) ---
print("Adding ML Risk Map layer...")
if risk_map is not None and map_bounds is not None:
    folium.raster_layers.ImageOverlay(
        image=risk_map,
        bounds=map_bounds,
        colormap=folium.LinearColormap(['green', 'yellow', 'red'], vmin=0, vmax=1).to_step(100),
        opacity=0.7,
        name='ML Wildfire Risk (India)'
    ).add_to(m)
else:
    print("Error: risk_map or map_bounds not available. Skipping risk layer.")

# --- D. Add Other Layers ---
add_live_fires(m)
add_pois(m)

# --- E. Add Final Touches ---
folium.LayerControl().add_to(m)

# --- F. Save to HTML ---
output_filename = 'map_with_layers_india.html'
m.save(output_filename)

print(f"SUCCESS: Map saved to {output_filename}")

# %% [markdown]
# ---
# ## ✅ Cell 9: Download Your Map!
# ---

# %%
# --- Cell 9: Download the File ---
print(f"Downloading {output_filename}...")
files.download(output_filename)


Creating the master Folium map...
Adding ML Risk Map layer...
Adding live fires layer...
Added 13768 live fire points globally.
Adding POIs layer...
SUCCESS: Map saved to map_with_layers_india.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>